In [2]:
import os


In [3]:
%pwd

'd:\\Data Science\\Git-Upload-Projects\\End_to_End_winequality\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\Data Science\\Git-Upload-Projects\\End_to_End_winequality'

In [6]:
import mlflow
# mlflow.end_run()  

In [7]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/singh.shanu1988/End_to_End_winequality.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="singh.shanu1988"
os.environ["MLFLOW_TRACKING_PASSWORD"]="85d3b3428bfdf23ee7d518a37c50b190e4e54914"


# MLFLOW_TRACKING_URI=https://dagshub.com/singh.shanu1988/End_to_End_winequality.mlflow \
# MLFLOW_TRACKING_USERNAME=singh.shanu1988 \
# MLFLOW_TRACKING_PASSWORD=85d3b3428bfdf23ee7d518a37c50b190e4e54914 \
# python script.py


In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [9]:
from winequality.constants import *
from winequality.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
     
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schemas_filepath = SCHEMA_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schemas_filepath)

            create_directories([self.config.dataStore_root])
        

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/singh.shanu1988/End_to_End_winequality.mlflow",
        )
        return model_evaluation_config

In [11]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
import urllib3

In [12]:
#mlflow.end_run()  
#! pip install urllib3  

In [13]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, pred):
        r2 = r2_score(actual, pred)
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        return r2, rmse, mae
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        x_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[self.config.target_column]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_artifact_uri()).scheme

        with mlflow.start_run():
            predict_quality = model.predict(x_test)
            (r2, rmse, mae) = self.eval_metrics(y_test, predict_quality)

            # Saving metrics as local
            scores = {"r2":r2, "rmse":rmse, "mae":mae}
            save_json(path=Path(self.config.metric_file_name), data=scores)
            
            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metrics("rmse", rmse)
            mlflow.log_metrics("r2", r2)
            mlflow.log_metrics("mae", mae)
            
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                
                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model,"model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [14]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    mlflow.end_run()
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e


[2024-04-12 23:23:16,635: INFO: common: YAML file loaded successfully: config\config.yaml]
[2024-04-12 23:23:16,637: INFO: common: YAML file loaded successfully: params.yaml]
[2024-04-12 23:23:16,640: INFO: common: YAML file loaded successfully: schema.yaml]
[2024-04-12 23:23:16,641: INFO: common: Created directory at: dataStore]
[2024-04-12 23:23:16,642: INFO: common: Created directory at: dataStore/model_evaluation]


Exception: Run with UUID a76af1872f524b91b48ab4b24d914f0d is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True

In [ ]:
try:
    config_manager = ConfigurationManager()  # Initialize ConfigurationManager
    model_evaluation_config = config_manager.get_model_evaluation_config()  # Get model evaluation configuration
    model_evaluation = ModelEvaluation(config=model_evaluation_config)  # Initialize ModelEvaluation

    # Start MLflow run
    model_evaluation.log_into_mlflow()

except Exception as e:
    raise e


[2024-04-10 00:41:31,454: INFO: common: YAML file loaded successfully: config\config.yaml]
[2024-04-10 00:41:31,457: INFO: common: YAML file loaded successfully: params.yaml]
[2024-04-10 00:41:31,460: INFO: common: YAML file loaded successfully: schema.yaml]
[2024-04-10 00:41:31,462: INFO: common: Created directory at: dataStore]
[2024-04-10 00:41:31,463: INFO: common: Created directory at: dataStore/model_evaluation]


Exception: Run with UUID fd613c08b97e4403bc777d780cc0d12a is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True

[2024-04-10 00:45:13,408: INFO: common: YAML file loaded successfully: config\config.yaml]
[2024-04-10 00:45:13,410: INFO: common: YAML file loaded successfully: params.yaml]
[2024-04-10 00:45:13,414: INFO: common: YAML file loaded successfully: schema.yaml]
[2024-04-10 00:45:13,416: INFO: common: Created directory at: dataStore]
[2024-04-10 00:45:13,417: INFO: common: Created directory at: dataStore/model_evaluation]


Exception: Run with UUID fd613c08b97e4403bc777d780cc0d12a is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True

In [ ]:
from pathlib import Path
from urllib.parse import urlparse
import mlflow
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import joblib

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schemas_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schemas_filepath)

        create_directories([self.config.dataStore_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/singh.shanu1988/End_to_End_winequality.mlflow",
        )
        return model_evaluation_config

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, pred):
        r2 = r2_score(actual, pred)
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        return r2, rmse, mae
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        x_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[self.config.target_column]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_artifact_uri()).scheme

        with mlflow.start_run():
            predict_quality = model.predict(x_test)
            (r2, rmse, mae) = self.eval_metrics(y_test, predict_quality)

            # Saving metrics as local
            scores = {"r2":r2, "rmse":rmse, "mae":mae}
            save_json(path=Path(self.config.metric_file_name), data=scores)
            
            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metrics("rmse", rmse)
            mlflow.log_metrics("r2", r2)
            mlflow.log_metrics("mae", mae)
            
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                mlflow.sklearn.log_model(model,"model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")
            
            mlflow.end_run()  # End the MLflow run

try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    
    # Check if there's an active MLflow run
    if mlflow.active_run():
        mlflow.end_run()

    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e


[2024-04-12 23:22:10,979: INFO: common: YAML file loaded successfully: config\config.yaml]
[2024-04-12 23:22:10,981: INFO: common: YAML file loaded successfully: params.yaml]
[2024-04-12 23:22:10,985: INFO: common: YAML file loaded successfully: schema.yaml]
[2024-04-12 23:22:10,986: INFO: common: Created directory at: dataStore]
[2024-04-12 23:22:10,988: INFO: common: Created directory at: dataStore/model_evaluation]


Exception: Run with UUID b3d084b107d84aad8aabbda67d4b928c is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True

In [15]:
from pathlib import Path
from urllib.parse import urlparse
import mlflow
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import joblib

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schemas_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schemas_filepath)

        create_directories([self.config.dataStore_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/singh.shanu1988/End_to_End_winequality.mlflow",
        )
        return model_evaluation_config

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, pred):
        r2 = r2_score(actual, pred)
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        return r2, rmse, mae
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        x_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[self.config.target_column]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_artifact_uri()).scheme

        # Start MLflow run
        with mlflow.start_run():
            predict_quality = model.predict(x_test)
            (r2, rmse, mae) = self.eval_metrics(y_test, predict_quality)

            # Saving metrics as local
            scores = {"r2":r2, "rmse":rmse, "mae":mae}
            save_json(path=Path(self.config.metric_file_name), data=scores)
            
            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metrics("rmse", rmse)
            mlflow.log_metrics("r2", r2)
            mlflow.log_metrics("mae", mae)
            
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                mlflow.sklearn.log_model(model,"model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    
    # Check if there's an active MLflow run
    if mlflow.active_run():
        mlflow.end_run()

    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e



[2024-04-12 23:23:19,336: INFO: common: YAML file loaded successfully: config\config.yaml]
[2024-04-12 23:23:19,340: INFO: common: YAML file loaded successfully: params.yaml]
[2024-04-12 23:23:19,344: INFO: common: YAML file loaded successfully: schema.yaml]
[2024-04-12 23:23:19,346: INFO: common: Created directory at: dataStore]
[2024-04-12 23:23:19,347: INFO: common: Created directory at: dataStore/model_evaluation]


Exception: Run with UUID 9544f92a99b04ad2afe48cef8c1fb975 is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True